In [1]:
from transformers import AutoTokenizer, LlamaForCausalLM
import transformers
import torch

In [2]:
model_name= "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Cuda available: ", torch.cuda.is_available())
model = LlamaForCausalLM.from_pretrained(
    model_name,
    device_map = "auto",
)

Cuda available:  True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [30]:
pairs_to_contrast =[("Red cockaded Woodpecker", "American Three toed Woodpecker"),
                    ("Louisiana Waterthrush", "Northern Waterthrush"),
                    ("Chuck will Widow","Nighthawk"),
                    ("Anna Hummingbird","Ruby throated Hummingbird"),
                    ("Artic Tern","Common Tern")]

In [23]:
def read_cub_classes():
    cub_classes = []
    with open('/export/scratch/ru86qer/datasets/cub_xxx_modified/CUB_200_2011/classes.txt', 'r') as f:
        cub_classes = [line.strip().split(' ')[1].split('.')[1] for line in f.readlines()]
    return cub_classes
cub_classes=read_cub_classes()
cub_classes

['Chuck_will_Widow',
 'Anna_Hummingbird',
 'Ruby_throated_Hummingbird',
 'Nighthawk',
 'Artic_Tern',
 'Common_Tern',
 'Northern_Waterthrush',
 'Louisiana_Waterthrush',
 'American_Three_toed_Woodpecker',
 'Red_cockaded_Woodpecker']

In [36]:
pairs_to_contrast_with_idx = []
for pair in pairs_to_contrast:
    pairs_to_contrast_with_idx.append((pair[0],cub_classes.index(pair[0].replace(' ','_')),[]))
    pairs_to_contrast_with_idx.append((pair[1],cub_classes.index(pair[1].replace(' ','_')),[]))
pairs_to_contrast_with_idx.sort(key=lambda x: x[1])
pairs_to_contrast_with_idx

[('Chuck will Widow', 0, []),
 ('Anna Hummingbird', 1, []),
 ('Ruby throated Hummingbird', 2, []),
 ('Nighthawk', 3, []),
 ('Artic Tern', 4, []),
 ('Common Tern', 5, []),
 ('Northern Waterthrush', 6, []),
 ('Louisiana Waterthrush', 7, []),
 ('American Three toed Woodpecker', 8, []),
 ('Red cockaded Woodpecker', 9, [])]

In [40]:
name_to_idx = {pair[0]:pair[1] for pair in pairs_to_contrast_with_idx}
name_to_idx

{'Chuck will Widow': 0,
 'Anna Hummingbird': 1,
 'Ruby throated Hummingbird': 2,
 'Nighthawk': 3,
 'Artic Tern': 4,
 'Common Tern': 5,
 'Northern Waterthrush': 6,
 'Louisiana Waterthrush': 7,
 'American Three toed Woodpecker': 8,
 'Red cockaded Woodpecker': 9}

In [4]:
prompt_index = 1
with open('/export/home/ru86qer/classify_by_description_release/prompts/prompt'+"_{0}.txt".format(prompt_index), 'r') as f:
    base_prompt = f.read()

In [5]:
prompts = []
for pair in pairs_to_contrast:
    prompts.append(base_prompt.format(category_name_1=pair[0], category_name_0=pair[1]))

In [6]:
responses=[]
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = inputs.to('cuda')
    # Generate
    generate_ids = model.generate(inputs.input_ids, max_length=len(inputs.input_ids[0])+700, top_k=1, num_return_sequences=1, do_sample=True)

    decoded_response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    response_text = decoded_response[(len(prompt)-7):]
    responses.append(response_text)

In [8]:
a = [response.split('\n') for response in responses]

In [49]:
b = [sentence.split(',') for sublist in a for sentence in sublist]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/export/home/ru86qer/miniconda3/envs/class/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_44801/2591420343.py", line 0, in <module>
  File "/tmp/ipykernel_44801/2591420343.py", line 0, in <module>
  File "_pydevd_bundle/pydevd_cython.pyx", line 1457, in _pydevd_bundle.pydevd_cython.SafeCallWrapper.__call__
  File "_pydevd_bundle/pydevd_cython.pyx", line 1834, in _pydevd_bundle.pydevd_cython.ThreadTracer.__call__
  File "_pydevd_bundle/pydevd_cython.pyx", line 1152, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 1135, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 312, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/export/home/ru86qer/miniconda3/envs/class/lib/python3.11/site-packages/debugpy/_ve

In [69]:
b

[[' While  Red cockaded Woodpeckers have a red cap on their head',
  ' American Three toed Woodpeckers have a black cap.'],
 ['- While Red cockaded Woodpeckers have a red nape',
  ' American Three toed Woodpeckers have a black nape.'],
 ['- While Red cockaded Woodpeckers have a red stripe on their face',
  ' American Three toed Woodpeckers have a black stripe on their face.'],
 ['- While Red cockaded Woodpeckers have a red patch on their cheek',
  ' American Three toed Woodpeckers have a black patch on their cheek.'],
 ['- While Red cockaded Woodpeckers have a red stripe on their back',
  ' American Three toed Woodpeckers have a black stripe on their back.'],
 ['- While Red cockaded Woodpeckers have a red tail',
  ' American Three toed Woodpeckers have a black tail.'],
 ['- While Red cockaded Woodpeckers have a red bill',
  ' American Three toed Woodpeckers have a black bill.'],
 ['- While Red cockaded Woodpeckers have a red legs',
  ' American Three toed Woodpeckers have black legs.']

In [50]:
name_to_idx.keys()

dict_keys(['Chuck will Widow', 'Anna Hummingbird', 'Ruby throated Hummingbird', 'Nighthawk', 'Artic Tern', 'Common Tern', 'Northern Waterthrush', 'Louisiana Waterthrush', 'American Three toed Woodpecker', 'Red cockaded Woodpecker'])

In [74]:
descriptor_dict = {}
for key in name_to_idx.keys():
    descriptor_dict[key] = []
    for sublist in b:
        for sentence in sublist:
            if key in sentence:
                if '- While' in sentence:
                    sentence_to_replace = sentence.replace('- While ','')
                    if sentence_to_replace not in descriptor_dict[key]:
                        descriptor_dict[key].append(sentence.replace('- While ',''))
                    continue
                if 'While ' in sentence:
                    sentence_to_replace = sentence.replace('While ','')
                    if sentence_to_replace not in descriptor_dict[key]:
                        descriptor_dict[key].append(sentence.replace('While ',''))
                    continue
                else:
                    if sentence not in descriptor_dict[key]:
                        descriptor_dict[key].append(sentence)

In [75]:
descriptor_dict

{'Chuck will Widow': ['  the wings of a Chuck will Widow are broad and rounded',
  'the body of a Chuck will Widow is large and bulky',
  'the tail of a Chuck will Widow is long and rounded',
  'the legs of a Chuck will Widow are thick and sturdy',
  'the eyes of a Chuck will Widow are large and round',
  'the mouth of a Chuck will Widow is wide and flat',
  'the color pattern of a Chuck will Widow is mottled and brown',
  'the shape of the wings of a Chuck will Widow is rounded',
  'the shape of the body of a Chuck will Widow is more rounded',
  'the shape of the tail of a Chuck will Widow is more rounded',
  'the shape of the legs of a Chuck will Widow is more sturdy',
  'the shape of the eyes of a Chuck will Widow is more rounded',
  'the shape of the mouth of a Chuck will Widow is more wide',
  'the shape of the color pattern of a Chuck will Widow is more mottled',
  'the shape of the wings of a Chuck will Widow is more rounded'],
 'Anna Hummingbird': ['  the Anna Hummingbird has a

In [76]:
import json
import os
path = "/export/home/ru86qer/classify_by_description_release/descriptors"
with open(os.path.join(path, "contrastive_descriptions.json"), 'w') as fp:
    json.dump(descriptor_dict, fp, indent=4)